In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [1]:
from datasets import load_dataset
data_files = {"train": "data.csv", "test": "test.csv"}
dataset = load_dataset("csv", data_files=data_files)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /Users/afaanansari/.cache/huggingface/datasets/csv/default-242651e72d291e48/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
dataset["test"][0]

{'text': '38] Zhong, X., Tang, J., Yepes, A.J.: Publaynet: largest dataset ever for doc-\nument layout analysis. In: 2019 International Conference on Document\nAnalysis and Recognition (ICDAR). pp. 1015–1022. IEEE (Sep 2019).\nhttps://doi.org/10.1109/ICDAR.2019.00166',
 'label': 0}

In [1]:
from datasets import load_dataset
imdb = load_dataset("imdb")
imdb["test"][0]

Found cached dataset imdb (/Users/afaanansari/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)


  0%|          | 0/3 [00:00<?, ?it/s]

{'text': 'I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn\'t match the background, and painfully one-dimensional characters cannot be overcome with a \'sci-fi\' setting. (I\'m sure there are those of you out there who think Babylon 5 is good sci-fi TV. It\'s not. It\'s clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It\'s really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it\'s rubbish as 

In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)
tokenized_dataset = dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/21 [00:00<?, ? examples/s]

Map:   0%|          | 0/19 [00:00<?, ? examples/s]

In [5]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
import evaluate
accuracy = evaluate.load("accuracy")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [6]:
import numpy as np
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [7]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [8]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'pre_classifier.weight', 'classifier.we

In [9]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 21
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 19
    })
})

In [12]:
training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
    # use_mps_device=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)
trainer.train()

/Users/afaanansari/Desktop/gtech/EVADB-chatgpt-filtering-data/test_evadb/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_loss': 0.5762024521827698, 'eval_accuracy': 1.0, 'eval_runtime': 0.9225, 'eval_samples_per_second': 20.596, 'eval_steps_per_second': 2.168, 'epoch': 1.0}


  0%|          | 0/2 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_loss': 0.2594030201435089, 'eval_accuracy': 1.0, 'eval_runtime': 5.1828, 'eval_samples_per_second': 3.666, 'eval_steps_per_second': 0.386, 'epoch': 9.0}


  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_loss': 0.25401973724365234, 'eval_accuracy': 1.0, 'eval_runtime': 4.6119, 'eval_samples_per_second': 4.12, 'eval_steps_per_second': 0.434, 'epoch': 10.0}
{'train_runtime': 500.6927, 'train_samples_per_second': 0.419, 'train_steps_per_second': 0.04, 'train_loss': 0.39654541015625, 'epoch': 10.0}


TrainOutput(global_step=20, training_loss=0.39654541015625, metrics={'train_runtime': 500.6927, 'train_samples_per_second': 0.419, 'train_steps_per_second': 0.04, 'train_loss': 0.39654541015625, 'epoch': 10.0})

In [23]:
trainer.push_to_hub()


OSError: Tried to clone a repository in a non-empty folder that isn't a git repository ('/Users/afaanansari/Desktop/gtech/EVADB-chatgpt-filtering-data/my_awesome_model'). If you really want to do this, do it manually:
 cd /Users/afaanansari/Desktop/gtech/EVADB-chatgpt-filtering-data/my_awesome_model && git init && git remote add origin && git pull origin main
 or clone repo to a new folder and move your existing files there afterwards.